In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
# from census import Census
from us import states
import json

#pip istall only needs to be ran once then this code can be removed
!pip install -U epiweeks
import epiweeks as ew

# Census API Key
from config import api_key
# c = Census(api_key)
c = api_key

In [18]:
#Disease CSV data: issues are traslating 'epiweek' column into a year, data frame will need to be shortened to only inlude years 2005-2010

disease_df = pd.read_csv("diseases_us_1916_to_2011.csv")

disease_df['disease'].value_counts()

C:\Users\fionn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


DIPHTHERIA     158983
MEASLES        145167
PERTUSSIS      109072
HEPATITIS A     90839
POLIO           81531
MUMPS           69754
RUBELLA         53205
SMALLPOX        50916
Name: disease, dtype: int64

In [19]:
disease_df["cases"]=disease_df["cases"].astype(str).replace("\\N",np.nan)
disease_df=disease_df.dropna(how="any")
disease_df['cases'] = disease_df["cases"].astype(int)

In [20]:
#Convert epi_week column into week and years columns.
disease_df['epi_week'] = disease_df['epi_week'].astype(str)
disease_df['year'] = disease_df['epi_week'].str[0:4]
disease_df['week'] = disease_df['epi_week'].str[4:6]
disease_df

,epi_week,state,loc,loc_type,disease,cases,incidence_per_100000,year,week
0,196601,MN,MINNESOTA,STATE,HEPATITIS A,3,0.08,1966,01
1,196601,CO,COLORADO,STATE,HEPATITIS A,1,0.05,1966,01
2,196601,AZ,ARIZONA,STATE,HEPATITIS A,6,0.37,1966,01
3,196601,MT,MONTANA,STATE,HEPATITIS A,2,0.28,1966,01
4,196601,LA,LOUISIANA,STATE,HEPATITIS A,1,0.03,1966,01
...,...,...,...,...,...,...,...,...,...
759462,194752,WI,SUPERIOR,CITY,DIPHTHERIA,0,0.00,1947,52
759463,194752,FL,TAMPA,CITY,DIPHTHERIA,0,0.00,1947,52
759464,194752,CO,DENVER,CITY,DIPHTHERIA,1,0.25,1947,52
759465,194752,NY,BUFFALO,CITY,DIPHTHERIA,2,0.35,1947,52


In [21]:
short_disease_df = disease_df.loc[(disease_df['year']== '2010')|(disease_df['year']== '2009')|(disease_df['year']== '2008')|(disease_df['year']== '2007')|(disease_df['year']== '2006')]

grouped_disease_df = short_disease_df.groupby(['year','state','disease',])
clean_disease_df=pd.DataFrame(grouped_disease_df['cases'].sum())
clean_disease_df

cases
year state disease           
2006 AK    PERTUSSIS       53
     AL    HEPATITIS A     17
           PERTUSSIS       90
     AR    HEPATITIS A     23
           PERTUSSIS       60
...                       ...
2010 WI    PERTUSSIS      304
     WV    HEPATITIS A      9
           PERTUSSIS       91
     WY    HEPATITIS A      3
           PERTUSSIS        6

[503 rows x 1 columns]

In [28]:
clean_disease_df= clean_disease_df.reset_index()
clean_disease_df

,year,state,disease,cases
0,2006,AK,PERTUSSIS,53
1,2006,AL,HEPATITIS A,17
2,2006,AL,PERTUSSIS,90
3,2006,AR,HEPATITIS A,23
4,2006,AR,PERTUSSIS,60
...,...,...,...,...
498,2010,WI,PERTUSSIS,304
499,2010,WV,HEPATITIS A,9
500,2010,WV,PERTUSSIS,91
501,2010,WY,HEPATITIS A,3


In [34]:
income_df['state recoded']=income_df['State'].replace (['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Dleaware','District of Columbia','Florida','Georgia','Hawii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Puerto Rico','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming'],['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','PR','RI','SC','SD','TN','TX','UT','VT','VA','WA','WA','WI','WY'])
income_df

,State,Population,Household Income,State Num,Year,state recoded
1,Alabama,4599030,38783,01,2006,AL
2,Alaska,670053,59393,02,2006,AK
3,Arizona,6166318,47265,04,2006,AZ
4,Arkansas,2810872,36599,05,2006,AR
5,California,36457549,56645,06,2006,CA
...,...,...,...,...,...,...
48,Washington,6744496,55631,53,2010,WA
49,West Virginia,1853973,38218,54,2010,WA
50,Wisconsin,5691047,49001,55,2010,WI
51,Wyoming,564460,53512,56,2010,WY


In [22]:
years=['2006','2007','2008','2009','2010']
All_data= {}
for year in years:
    try:
        target_url = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01003_001E,B19013_001E&for=state:*&key={api_key}"
        income_data = requests.get(target_url).json()
        All_data.update({year:income_data})
    except:
        print ('not found')
        
All_data

{'2006': [['NAME', 'B01003_001E', 'B19013_001E', 'state'],
  ['Alabama', '4599030', '38783', '01'],
  ['Alaska', '670053', '59393', '02'],
  ['Arizona', '6166318', '47265', '04'],
  ['Arkansas', '2810872', '36599', '05'],
  ['California', '36457549', '56645', '06'],
  ['Colorado', '4753377', '52015', '08'],
  ['Connecticut', '3504809', '63422', '09'],
  ['Delaware', '853476', '52833', '10'],
  ['District of Columbia', '581530', '51847', '11'],
  ['Florida', '18089889', '45495', '12'],
  ['Georgia', '9363941', '46832', '13'],
  ['Hawaii', '1285498', '61160', '15'],
  ['Idaho', '1466465', '42865', '16'],
  ['Illinois', '12831970', '52006', '17'],
  ['Indiana', '6313520', '45394', '18'],
  ['Iowa', '2982085', '44491', '19'],
  ['Kansas', '2764075', '45478', '20'],
  ['Kentucky', '4206074', '39372', '21'],
  ['Louisiana', '4287768', '39337', '22'],
  ['Maine', '1321574', '43439', '23'],
  ['Maryland', '5615727', '65144', '24'],
  ['Massachusetts', '6437193', '59963', '25'],
  ['Michigan', 

In [23]:
#reading the census Json data into a dataframe. issues include translating the state names into state abreviations to match the disease data frame, dropping the frst row, and renaming the columns
# income_df = pd.DataFrame(All_data)

income_df= pd.DataFrame()
years=['2006','2007','2008','2009','2010']
for year in years:
    df= pd.DataFrame(All_data[year])
    df['year']=year
    df.drop(index=df.index[0],axis=0,inplace=True)
    income_df=pd.concat([income_df,df])
income_df.columns= ['State', 'Population', 'Household Income', 'State Num', 'Year']
income_df

,State,Population,Household Income,State Num,Year
1,Alabama,4599030,38783,01,2006
2,Alaska,670053,59393,02,2006
3,Arizona,6166318,47265,04,2006
4,Arkansas,2810872,36599,05,2006
5,California,36457549,56645,06,2006
...,...,...,...,...,...
48,Washington,6744496,55631,53,2010
49,West Virginia,1853973,38218,54,2010
50,Wisconsin,5691047,49001,55,2010
51,Wyoming,564460,53512,56,2010


In [38]:
# Merging dataframes
df_merged= pd.merge(clean_disease_df,income_df,left_on='state',right_on='state recoded',how='inner') 
df_merged
df_merged=df_merged.drop(columns=['state recoded','State'])
df_merged

,year,state,disease,cases,Population,Household Income,State Num,Year
0,2006,AK,PERTUSSIS,53,670053,59393,02,2006
1,2006,AK,PERTUSSIS,53,683478,64333,02,2007
2,2006,AK,PERTUSSIS,53,686293,68460,02,2008
3,2006,AK,PERTUSSIS,53,698473,66953,02,2009
4,2006,AK,PERTUSSIS,53,713985,64576,02,2010
...,...,...,...,...,...,...,...,...
2410,2010,WY,PERTUSSIS,6,515004,47423,56,2006
2411,2010,WY,PERTUSSIS,6,522830,51731,56,2007
2412,2010,WY,PERTUSSIS,6,532668,53207,56,2008
2413,2010,WY,PERTUSSIS,6,544270,52664,56,2009
